Analiza Wyników Matur z 2023
=========================

Kod pozwalający odtworzyć wyniki z analizy matur z roku 2023.

Aby uruchomić kod, należy oprócz skonfigurowania repozytorium (patrz README.md) przygotować następujące pliki z wynikami:
- matura-2023.tsv
- matura-2022.tsv
- matura-2021.tsv


Pliki te można pobrać ze strony: https://mapa.wyniki.edu.pl/MapaEgzaminow/ wybierając:
- Dla wyników z roku 2023:
  - Rodzaj deklaracji: Egzamin maturalny w formule 2023
  - Rok 2023
  - Kliknąć: Pobieranie plików z wynikami -> EM2023 - szkoły (aktualizacja 07.2023).xlsx
- Dla wyników z roku 2021 i 2022:
  - Rodzaj deklaracji: Egzamin maturalny w formule 2015
  - Rok 2021 lub 2022
  - Kliknąć: Pobieranie plików z wynikami -> EM2015 - szkoły (aktualizacja 07.2021).xlsx lub EM2015 - szkoły (aktualizacja 07.2022).xlsx

Pobrane pliki będą w formacie Excel. Należy je otworzyć i przekonwertować do formatu CSV/TSV:
- Separator pola: {tabulator}
- Ogranicznik ciągu: brak

Tak przygotowane pliki należy przegrać do katalogu:

```
{ścieżka w której leży repozytorium}/ai-polit/resources/matura
```

W momencie analizy dane na stronie były zaktualizowane w dniu: 04-07-2023

Data przeprowadzenia analizy: 22-07-2023


In [61]:
import os
import openpyxl
import re
from tqdm import tqdm
import logging
logging.basicConfig(level=logging.DEBUG, format='[%(levelname)s] %(asctime)s\t%(message)s')

In [96]:
from collections import OrderedDict
from collections import Counter

In [110]:
# ładne drukowanie tabelek
import pandas as pd
from IPython.display import display, HTML

def show_pretty_table(raw_data, header):
    df = pd.DataFrame(raw_data, columns=header)
    display(HTML(df.to_html()))

In [191]:
RESOURCES_DATA_DIR = '../resources/matura'
ANALYZED_YEARS = [2021, 2022, 2023]
#ANALYZED_YEARS = [2021, 2022]

ANALYZED_LAST_YEAR = ANALYZED_YEARS[-1]
SUBHEADER_SEPARATOR = '===='

Przygotowanie danych
====================

In [21]:
for year in ANALYZED_YEARS:
    fn = f"matura-{year}.xlsx" 
    assert fn in os.listdir(RESOURCES_DATA_DIR), f"Brakujący plik z wynikami matur = {fn}. Sprawdź jak przygotować pliki na początku tego notebooka."

In [208]:
def parse_header(row, unify=True):
    raw = []
    for cell_val in row:
        if cell_val is not None and isinstance(cell_val, str):
            cell_val = re.sub("^\s+", "", cell_val)
            cell_val = re.sub("\s+$", "", cell_val)
            cell_val = re.sub(r"\s+", " ", cell_val)
            if unify:
                # trochę dodatkowej unifikacji nagłówków
                cell_val = re.sub(" - ", " ", cell_val)
                cell_val = re.sub(r"\s*[(][mM][)]\s*$", "", cell_val)
                
                cell_val = re.sub("^\s+", "", cell_val)
                cell_val = re.sub("\s+$", "", cell_val)
                cell_val = re.sub(r"\s+", " ", cell_val)

        raw.append(cell_val)
    return raw

def merge_raw_headers(raw1, raw2):
    merged = []
    header_prefix = None
    duplicated_count = 0
    for i2, el2 in enumerate(raw2):
        if el2 is None:
            break
            
        cur_el1 = None
        if i2 < len(raw1):
            cur_el1 = raw1[i2]
        
        if cur_el1 is not None:
            header_prefix = cur_el1
        
        final_header_val = el2
        if header_prefix is not None:
            final_header_val = f"{header_prefix}{SUBHEADER_SEPARATOR}{el2}"
            
        final_header_val = final_header_val.lower()
        if final_header_val in merged:
            duplicated_count += 1
            final_header_val = f"duplicated{duplicated_count}"
        merged.append(final_header_val)
        
    return merged


def create_matura_results_entry(header, raw_row):
    entry = OrderedDict()
    for i, key in enumerate(header):
        value = None
        if i < len(raw_row):
            value = raw_row[i]
        entry[key] = value
    return entry
    
    
def parse_worksheet(worksheet):
    raw_header1, raw_header2 = [], []
    header = []
    parsed_data = []
   
    for row_i, row in tqdm(enumerate(worksheet.iter_rows(min_row=None, max_row=worksheet.max_row, min_col=None, max_col=None, values_only=True))):
        if row_i == 0:
            continue
        elif row_i == 1:
            raw_header1 = parse_header(row)
        elif row_i == 2:
            raw_header2 = parse_header(row)
            header = merge_raw_headers(raw_header1, raw_header2)
        else:
            raw_row = parse_header(row, unify=False)            
            entry = create_matura_results_entry(header, raw_row)
            parsed_data.append(entry)
    return parsed_data

def parse_matura_results(fp):    
    workbook = openpyxl.load_workbook(fp)
    worksheet = workbook.active
    parsed_data = parse_worksheet(worksheet)
    return parsed_data

In [209]:
matura_year_to_results = OrderedDict()

for year in ANALYZED_YEARS:    
    fp = os.path.join(RESOURCES_DATA_DIR, f"matura-{year}.xlsx")
    
    logging.info("Start parsing: %s", fp)
    parsed_data = parse_matura_results(fp)
    matura_year_to_results[year] = parsed_data
    logging.info("Finished parsing: %s", fp)
    

[INFO] 2023-07-23 02:01:56,046	Start parsing: ../resources/matura/matura-2021.xlsx
4892it [00:03, 1460.71it/s]
[INFO] 2023-07-23 02:02:14,752	Finished parsing: ../resources/matura/matura-2021.xlsx
[INFO] 2023-07-23 02:02:14,752	Start parsing: ../resources/matura/matura-2022.xlsx
4825it [00:06, 708.46it/s]
[INFO] 2023-07-23 02:02:40,978	Finished parsing: ../resources/matura/matura-2022.xlsx
[INFO] 2023-07-23 02:02:40,978	Start parsing: ../resources/matura/matura-2023.xlsx
2325it [00:03, 706.97it/s]
[INFO] 2023-07-23 02:03:02,597	Finished parsing: ../resources/matura/matura-2023.xlsx


In [215]:
# Kilka losowych sprawdzeń czy wszystko jest wgrane poprawnie
# Sprawdzamy na ostatnich wierszach z Excela
# Uwaga, jeśli dane z 2023 zostaną zaktualizowane tutaj należy zmienić indeks test_entry_2023 
# na numer wiersza szkoły o RSPO = 85757 odejmując od niego 4

test_entry_2021 = matura_year_to_results[2021][4888]
test_entry_2022 = matura_year_to_results[2022][4821]
test_entry_2023 = matura_year_to_results[2023][2321]

assert test_entry_2021['województwo nazwa'] == 'Opolskie'
assert test_entry_2021['czy publiczna'] == 'Tak'
assert test_entry_2021['rspo szkoły'] == 85757
assert test_entry_2021['język polski poziom rozszerzony====* liczba zdających'] == 8
assert test_entry_2021['język polski poziom rozszerzony====średni wynik (%)'] == ''
assert test_entry_2021['język polski poziom podstawowy====* liczba zdających'] == 15
assert abs(test_entry_2021['język polski poziom podstawowy====średni wynik (%)'] - 64.33) < 0.01

assert test_entry_2021['matematyka poziom rozszerzony====* liczba zdających'] == ''
assert test_entry_2021['matematyka poziom rozszerzony====średni wynik (%)'] == ''
assert test_entry_2021['matematyka poziom podstawowy====* liczba zdających'] == 15
assert abs(test_entry_2021['matematyka poziom podstawowy====średni wynik (%)'] - 41.0666666666667) < 0.01

assert test_entry_2022['województwo nazwa'] == 'Opolskie'
assert test_entry_2022['czy publiczna'] == 'Tak'
assert test_entry_2022['rspo szkoły'] == 56036
assert test_entry_2022['język polski poziom rozszerzony====* liczba zdających'] == 1
assert test_entry_2022['język polski poziom rozszerzony====średni wynik (%)'] == ''
assert test_entry_2022['język polski poziom podstawowy====* liczba zdających'] == 16
assert abs(test_entry_2022['język polski poziom podstawowy====średni wynik (%)'] - 48.875) < 0.01

assert test_entry_2022['matematyka poziom rozszerzony====* liczba zdających'] == 5
assert test_entry_2022['matematyka poziom rozszerzony====średni wynik (%)'] == 17.6
assert test_entry_2022['matematyka poziom podstawowy====* liczba zdających'] == 16
assert abs(test_entry_2022['matematyka poziom podstawowy====średni wynik (%)'] - 51.5) < 0.01

assert test_entry_2023['województwo nazwa'] == 'Opolskie'
assert test_entry_2023['czy publiczna'] == 'Tak'
assert test_entry_2023['rspo szkoły'] == 85757
assert test_entry_2023['język polski poziom rozszerzony====* liczba zdających'] == 4
assert test_entry_2023['język polski poziom rozszerzony====średni wynik (%)'] == ''
assert test_entry_2023['język polski poziom podstawowy====* liczba zdających'] == 19
assert abs(test_entry_2023['język polski poziom podstawowy====średni wynik (%)'] - 61.6842105263158) < 0.01

assert test_entry_2023['matematyka poziom rozszerzony====* liczba zdających'] == 3
assert test_entry_2023['matematyka poziom rozszerzony====średni wynik (%)'] == ''
assert test_entry_2023['matematyka poziom podstawowy====* liczba zdających'] == 19
assert abs(test_entry_2023['matematyka poziom podstawowy====średni wynik (%)'] - 58.6315789473684) < 0.01

Przygotowanie danych do analizy
================================

In [216]:
matura_rspo_to_year_results = dict()

for year in ANALYZED_YEARS:
    results = matura_year_to_results[year]
    for res in results:
        rspo = res['rspo szkoły']
        if rspo not in matura_rspo_to_year_results:
            matura_rspo_to_year_results[rspo] = dict()
        matura_rspo_to_year_results[rspo][year] = res

In [217]:
# Teraz ograniczymy się tylko do szkół, które mają dostępne dane za wszystkie trzy lata
# połączymy je za pomocą kolumny RSPO
# Uwaga: na dalszych etapach ten zbiór się jeszcze zawęzi, bo nie wszystkie szkoły
# mają dane dla wszystkich przedmiotów (ze względu na anonimizcję)

def create_schools_with_all_data_available():
    """
    Returns: set of RSPO of schools for which all data is available
    """
    rspo_counter = Counter()
    for year in ANALYZED_YEARS:
        results = matura_year_to_results[year]
        for res in results:
            rspo = res['rspo szkoły']
            rspo_counter[rspo] += 1
            
    rspo_set = set()
    for rspo, freq in rspo_counter.items():
        if freq == len(ANALYZED_YEARS):
            rspo_set.add(rspo)
            
    return rspo_set
    

all_data_schools_rspo_set = create_schools_with_all_data_available()
logging.info("Mamy %i szkół dla których są dane ze wszystkich lat %s", len(all_data_schools_rspo_set), ANALYZED_YEARS)

[INFO] 2023-07-23 02:05:32,878	Mamy 2084 szkół dla których są dane ze wszystkich lat [2021, 2022, 2023]


In [218]:
def check_if_all_results_are_available_for_rspo(rspo, subject):
    for year in ANALYZED_YEARS:
        year_results = matura_rspo_to_year_results[rspo][year]
        for level in ['poziom podstawowy', 'poziom rozszerzony']:
            key_prefix = f"{subject} {level}"
            for key_val in ['* liczba zdających', 'średni wynik (%)']:
                final_key = f"{key_prefix}{SUBHEADER_SEPARATOR}{key_val}"
                value = year_results[final_key]
                if value is None:
                    return False
                if value == '':
                    return False
                if value <= 0.000001:
                    return False
    return True
                    

def create_schools_for_analysis(subject, filter_wojewodztwo=None, filter_gmina=None, filter_rodzaj=None):
    analyzed_rspo_set = set()
    # filter geographically
    for rspo in all_data_schools_rspo_set:
        if ANALYZED_LAST_YEAR not in matura_rspo_to_year_results[rspo]:
            print(matura_rspo_to_year_results[rspo])
            
        last_res = matura_rspo_to_year_results[rspo][ANALYZED_LAST_YEAR]
        
        if filter_wojewodztwo is not None:
            if filter_wojewodztwo != last_res['województwo nazwa']:
                continue
        if filter_gmina is not None:
            if filter_gmina != last_res['gmina nazwa']:
                continue
        if filter_rodzaj is not None:
            if filter_rodzaj != last_res['rodzaj placówki']:
                continue
        analyzed_rspo_set.add(rspo)
    
    # now filter those for which all results are available for given subject
    all_results_analyzed_rspo_set = set()
    for rspo in analyzed_rspo_set:
        if check_if_all_results_are_available_for_rspo(rspo, subject):
            all_results_analyzed_rspo_set.add(rspo)
    return all_results_analyzed_rspo_set


def do_compute_avg_for_schools(rspo_set, year, subject, level):
    total_result_val = 0
    total_zdajacy = 0
    
    key_prefix = f"{subject} {level}"
    for rspo in rspo_set:
        year_results = matura_rspo_to_year_results[rspo][year]
        rspo_avg_result = year_results[f"{key_prefix}{SUBHEADER_SEPARATOR}średni wynik (%)"]
        rspo_zdajacy = year_results[f"{key_prefix}{SUBHEADER_SEPARATOR}* liczba zdających"]
        rspo_res_value = rspo_avg_result * rspo_zdajacy
        total_result_val += rspo_res_value
        total_zdajacy += rspo_zdajacy
    total_avg = total_result_val / total_zdajacy
    
    assert total_avg >= 0.0
    
    return total_avg
    
    
def filter_public_rspo(rspo_set, public):
    public_value = 'tak'
    if not public:
        public_value = 'nie'
    
    result_set = set()
    for rspo in rspo_set:
        last_res = matura_rspo_to_year_results[rspo][ANALYZED_LAST_YEAR]
        czy_publiczna_value = last_res['czy publiczna'].lower()
        if czy_publiczna_value == public_value:
            result_set.add(rspo)
    return result_set
        
    
def compute_avg_results_from_matura(subject, filter_wojewodztwo=None, filter_gmina=None, filter_rodzaj='dla młodzieży'):
    analyzed_schools_rspo_set = create_schools_for_analysis(subject, filter_wojewodztwo, filter_gmina, filter_rodzaj)
    logging.info("Analizuję przedmiot: %s filtry(województwo=%s, gmina=%s, rodzaj=%s), liczba dopasowanych szkół: %i",
                 subject,
                 filter_wojewodztwo,
                 filter_gmina,
                 filter_rodzaj,
                 len(analyzed_schools_rspo_set),
                )
    
    analyzed_schools_rspo_set_public = filter_public_rspo(analyzed_schools_rspo_set, public=True)
    analyzed_schools_rspo_set_nonpublic = filter_public_rspo(analyzed_schools_rspo_set, public=False)
    
    logging.info("Szkoły publiczne: %i, Szkoły niepubliczne: %i",
                 len(analyzed_schools_rspo_set_public),
                len(analyzed_schools_rspo_set_nonpublic))
    
    public_row = ['Publiczne', len(analyzed_schools_rspo_set_public)]
    nonpublic_row = ['Niepubliczne', len(analyzed_schools_rspo_set_nonpublic)]
    all_row = ['Wszystkie', len(analyzed_schools_rspo_set)]
    
    for year in ANALYZED_YEARS:
        level = 'poziom podstawowy'
        public_avg = do_compute_avg_for_schools(analyzed_schools_rspo_set_public, year, subject, level)
        nonpublic_avg = do_compute_avg_for_schools(analyzed_schools_rspo_set_nonpublic, year, subject, level)
        all_avg = do_compute_avg_for_schools(analyzed_schools_rspo_set, year, subject, level)
        
        public_row.append(public_avg)
        nonpublic_row.append(nonpublic_avg)
        all_row.append(all_avg)

    for year in ANALYZED_YEARS:
        level = 'poziom rozszerzony'
        public_avg = do_compute_avg_for_schools(analyzed_schools_rspo_set_public, year, subject, level)
        nonpublic_avg = do_compute_avg_for_schools(analyzed_schools_rspo_set_nonpublic, year, subject, level)
        all_avg = do_compute_avg_for_schools(analyzed_schools_rspo_set, year, subject, level)
        
        public_row.append(public_avg)
        nonpublic_row.append(nonpublic_avg)
        all_row.append(all_avg)
        
    final_table = [public_row, nonpublic_row, all_row]
    header = ['Typ szkoły', 'L. szkół']
    for year in ANALYZED_YEARS:
        header.append(f"{year}: śr. (podst)")
    for year in ANALYZED_YEARS:
        header.append(f"{year}: śr. (roz)")
        
    show_pretty_table(final_table, header)
    
    return final_table    

In [219]:
for subject in ['język polski', 'matematyka']:
    final_table = compute_avg_results_from_matura(subject, filter_gmina='Warszawa')

[INFO] 2023-07-23 02:05:46,813	Analizuję przedmiot: język polski filtry(województwo=None, gmina=Warszawa, rodzaj=dla młodzieży), liczba dopasowanych szkół: 95
[INFO] 2023-07-23 02:05:46,814	Szkoły publiczne: 81, Szkoły niepubliczne: 14


,Typ szkoły,L. szkół,2021: śr. (podst),2022: śr. (podst),2023: śr. (podst),2021: śr. (roz),2022: śr. (roz),2023: śr. (roz)
0,Publiczne,81,64.433450,60.642953,72.379996,59.611020,66.159989,56.702826
1,Niepubliczne,14,63.422829,61.720116,69.047679,61.769644,67.433820,50.940171
2,Wszystkie,95,64.370032,60.722680,72.157599,59.777919,66.282875,56.200081


[INFO] 2023-07-23 02:05:46,823	Analizuję przedmiot: matematyka filtry(województwo=None, gmina=Warszawa, rodzaj=dla młodzieży), liczba dopasowanych szkół: 79
[INFO] 2023-07-23 02:05:46,824	Szkoły publiczne: 68, Szkoły niepubliczne: 11


,Typ szkoły,L. szkół,2021: śr. (podst),2022: śr. (podst),2023: śr. (podst),2021: śr. (roz),2022: śr. (roz),2023: śr. (roz)
0,Publiczne,68,79.370429,79.645344,83.793735,53.814910,58.442178,64.265919
1,Niepubliczne,11,73.062878,71.974010,75.324965,42.813871,48.283894,57.693333
2,Wszystkie,79,78.999049,79.076210,83.222833,53.250754,57.739904,63.868944


In [220]:
for subject in ['język polski', 'matematyka']:
    final_table = compute_avg_results_from_matura(subject)

[INFO] 2023-07-23 02:07:27,354	Analizuję przedmiot: język polski filtry(województwo=None, gmina=None, rodzaj=dla młodzieży), liczba dopasowanych szkół: 1062
[INFO] 2023-07-23 02:07:27,359	Szkoły publiczne: 1006, Szkoły niepubliczne: 56


,Typ szkoły,L. szkół,2021: śr. (podst),2022: śr. (podst),2023: śr. (podst),2021: śr. (roz),2022: śr. (roz),2023: śr. (roz)
0,Publiczne,1006,62.232495,60.523163,67.883337,53.488920,60.627187,51.609290
1,Niepubliczne,56,61.322385,59.814006,65.536153,54.291473,60.654175,50.307861
2,Wszystkie,1062,62.212144,60.503970,67.824996,53.508943,60.627937,51.574904


[INFO] 2023-07-23 02:07:27,395	Analizuję przedmiot: matematyka filtry(województwo=None, gmina=None, rodzaj=dla młodzieży), liczba dopasowanych szkół: 889
[INFO] 2023-07-23 02:07:27,397	Szkoły publiczne: 834, Szkoły niepubliczne: 55


,Typ szkoły,L. szkół,2021: śr. (podst),2022: śr. (podst),2023: śr. (podst),2021: śr. (roz),2022: śr. (roz),2023: śr. (roz)
0,Publiczne,834,72.119942,73.060161,77.337257,41.832875,46.622925,52.935016
1,Niepubliczne,55,74.373167,73.003674,77.408069,44.563942,49.812376,59.246377
2,Wszystkie,889,72.172843,73.058511,77.339252,41.904099,46.726034,53.130859


In [221]:
for subject in ['język polski', 'matematyka']:
    final_table = compute_avg_results_from_matura(subject, filter_gmina='Kraków')

[INFO] 2023-07-23 02:08:15,041	Analizuję przedmiot: język polski filtry(województwo=None, gmina=Kraków, rodzaj=dla młodzieży), liczba dopasowanych szkół: 39
[INFO] 2023-07-23 02:08:15,042	Szkoły publiczne: 33, Szkoły niepubliczne: 6


,Typ szkoły,L. szkół,2021: śr. (podst),2022: śr. (podst),2023: śr. (podst),2021: śr. (roz),2022: śr. (roz),2023: śr. (roz)
0,Publiczne,33,64.457763,63.552762,72.367866,57.216552,65.282855,62.192065
1,Niepubliczne,6,63.192586,61.634915,73.034186,70.074118,71.383333,61.827869
2,Wszystkie,39,64.373328,63.404682,72.424633,57.947584,65.754237,62.159821


[INFO] 2023-07-23 02:08:15,050	Analizuję przedmiot: matematyka filtry(województwo=None, gmina=Kraków, rodzaj=dla młodzieży), liczba dopasowanych szkół: 32
[INFO] 2023-07-23 02:08:15,050	Szkoły publiczne: 27, Szkoły niepubliczne: 5


,Typ szkoły,L. szkół,2021: śr. (podst),2022: śr. (podst),2023: śr. (podst),2021: śr. (roz),2022: śr. (roz),2023: śr. (roz)
0,Publiczne,27,77.380584,79.111241,83.312773,50.509717,56.137392,62.267258
1,Niepubliczne,5,78.193577,81.329605,84.645067,49.590909,60.270270,68.130435
2,Wszystkie,32,77.432565,79.290871,83.427068,50.449167,56.524719,62.773511
